In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from time import time_ns

In [ ]:
def forceExt(time, freqDrive):
    return F0 * np.cos((freqDrive*2*np.pi) * time)


def derivL(y, v, stap, fext):
    y += [y[-1] + stap * (v[-1])]
    v += [v[-1] + (fext) - (c1 * y[-2]) - (c2 * v[-1])]
    return y, v

tstep = 8e-7
tstart = 0
tstartMotor = 1
tstop = 20
t = np.arange(tstart, tstartMotor, tstep)

v = [0.]
y = [0.]
y += [y[0]+v[0]*tstep]

F0 = 5e-3
freqDrive = 800
freqRes = 800

mass = 7.86e-3 # kg
gamma_sys = 5
stiffness = 2.10e5 # N/m
# height = 0
# hv = 0.01
om_sq = (np.pi * 2 * (freqRes))**2
c0 = tstep / mass
c1 = tstep*om_sq
c2 = c0 * gamma_sys
t1 = time_ns()
for i in t:
    y, v = derivL(y, v, tstep, c0 * forceExt(i, freqDrive))
    

t = np.arange(tstartMotor, tstop, tstep)
for i in t:
    freqDrive = 800 + 5* np.cos(np.pi/4*i)
    y, v = derivL(y, v, tstep, c0 * forceExt(i, freqDrive))
    # if not np.isfinite(y[-1]):
    #     y = y[:-11]
    #     v = v[:-10]
    #     break
    # if not np.isfinite(v[-1]):
    #     y = y[:-10]
    #     v = v[:-11]
    #     break

tend = (time_ns()-t1)*1e-9
print(tend, " s")
print((tstop-tstart)/tend, " efficiency")

t = np.arange(tstart, tstop, tstep)


In [ ]:
fig = plt.figure(figsize=(10,4))
ax1 = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)
ax1.plot(t[:len(y)-2], y[2:], c='blue')
ax2.plot(t[:len(v)-1], v[1:], c='orange')
# plt.legend()
plt.show()

Other old code that might be usefull maybe

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#Berekend de differenciaal vergelijking op de euler methode
def calc_dif_vergelijking(x0, x1, time, h, massa, omg_sq, gamma_sys, gamma_vis, freq, Fm):
    #Berekend de constanten tijdens de dffernciaal vergelijking
    gamma = gamma_sys + gamma_vis
    
    veerconstante = omg_sq * massa
    a = (4*massa - (2*(h**2)*veerconstante))/(2*massa + h*gamma)
    b = (h*gamma - 2*massa)/(2*massa + h*gamma)
    c = Fm*(2*(h**2))/(2*massa + h*gamma)

    x = [x0, x1]

    i = 1
    #Berekend elke stap
    while i < len(time) - 1:
        x += [x[i]*a + x[i-1]*b + np.sin(2*np.pi*freq*time[i]) * c]
        i += 1

    return x

In [ ]:
# const
mass = 1
b = 0.1
veerconstante = 0.01
F0 = 1
freq = 0.1
om_sq = veerconstante/mass
gam_sys = b/mass

# tijd
teind = 100
Nstap = 1000
tijd = np.linspace(0, teind, Nstap + 1)
dt = teind / (Nstap)

F0e = F0 / (mass / (dt**2) + b / (2 * dt))
x0 = 0
v0 = 0

x = np.zeros_like(tijd)
x[0] = x0
x[1] = x0 + dt * v0



for ti in range(1, Nstap):
    a = (om_sq - 2 / (dt**2)) / (mass / (dt**2) + (b / 2 * dt)/mass)
    b = (mass / (dt**2)) - b / (2 * dt) / (mass / (dt**2) + b / (2 * dt))
    x[ti + 1] = -a * x[ti] - b * x[ti - 1] + F0e * np.sin(2 * np.pi * freq * tijd[ti])
    if not np.isfinite(x[ti + 1]):
        print(f"Reached infinity!\nIndex: {ti}, time: {tijd[ti]}")
        x = x[:ti]
        tijd = tijd[:ti]
        break
plt.plot(tijd, x)
plt.yscale('log')
plt.show()

In [ ]:
# const
mass = 1
b = 0.1
veerconstante = 0.01
F0 = 1
freq = 2
freq_res= 1
om_sq = np.pi * 2 * freq_res
gam_sys = b/mass

# tijd
teind = 100
Nstap = 10000

tijd = np.linspace(0, teind, Nstap + 1)
dt = teind / (Nstap)

x0 = 0

x= calc_dif_vergelijking(x0=x0, x1=x0+dt*(F0*np.cos(2*np.pi*freq*tijd[1])), time=tijd, h=dt, massa=mass, omg_sq=om_sq, gamma_sys=gam_sys, gamma_vis=0, freq=freq, Fm=F0)
plt.plot(x)